# 1

In [4]:
import pandas as pd

# train = pd.read_csv(r'C:/Users/DC/OneDrive - 계명대학교/DC/2024/2024_날씨빅콘/데셋/train_fi.csv')
test = pd.read_csv(r'C:/Users/DC/OneDrive - 계명대학교/DC/2024/2024_날씨빅콘/데셋/test_fi.csv')

In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7593181 entries, 0 to 7593180
Data columns (total 93 columns):
 #   Column         Dtype  
---  ------         -----  
 0   격자넘버           int64  
 1   시간             int64  
 2   지점번호           int64  
 3   기온             float64
 4   상대습도           float64
 5   풍속             float64
 6   강수량            float64
 7   체감온도           float64
 8   요일             int64  
 9   주중주말           int64  
 10  전력기상지수         float64
 11  계절             object 
 12  년              int64  
 13  월              int64  
 14  일              int64  
 15  기온_mean_num    float64
 16  기온_std_num     float64
 17  기온_skew_num    float64
 18  상대습도_mean_num  float64
 19  상대습도_std_num   float64
 20  상대습도_skew_num  float64
 21  풍속_mean_num    float64
 22  풍속_std_num     float64
 23  풍속_skew_num    float64
 24  강수량_mean_num   float64
 25  강수량_std_num    float64
 26  강수량_skew_num   float64
 27  체감온도_mean_num  float64
 28  체감온도_std_num   float64
 29  체감온도_skew_num 

In [3]:
object_columns = train.select_dtypes(include=['object']).columns
print("Object 타입의 열:", object_columns)
train = pd.get_dummies(train, columns=object_columns)

Object 타입의 열: Index(['계절'], dtype='object')


In [9]:
# boolean 값을 정수형으로 변환
train = train.astype(float)

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2838143 entries, 0 to 2838142
Data columns (total 92 columns):
 #   Column         Dtype  
---  ------         -----  
 0   격자넘버           int64  
 1   시간             int64  
 2   지점번호           int64  
 3   기온             float64
 4   상대습도           float64
 5   풍속             float64
 6   강수량            float64
 7   체감온도           float64
 8   요일             int64  
 9   주중주말           int64  
 10  계절             object 
 11  년              int64  
 12  월              int64  
 13  일              int64  
 14  기온_mean_num    float64
 15  기온_std_num     float64
 16  기온_skew_num    float64
 17  상대습도_mean_num  float64
 18  상대습도_std_num   float64
 19  상대습도_skew_num  float64
 20  풍속_mean_num    float64
 21  풍속_std_num     float64
 22  풍속_skew_num    float64
 23  강수량_mean_num   float64
 24  강수량_std_num    float64
 25  강수량_skew_num   float64
 26  체감온도_mean_num  float64
 27  체감온도_std_num   float64
 28  체감온도_skew_num  float64
 29  기온_mean_bn    

# 2

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# train 데이터프레임이 이미 정의되어 있어야 합니다.
# 예시: train = pd.read_csv('train.csv')

X = train.drop('전력기상지수', axis=1)
y = train['전력기상지수']

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 목표 변수 numpy array로 변환 후 reshape
y_train_np = y_train.values.reshape(-1, 1)
y_val_np = y_val.values.reshape(-1, 1)

# TabNet 모델 생성
tabnet = TabNetRegressor()

# 랜덤 서치 파라미터 설정
param_dist = {
    'n_d': [8, 16, 24, 32, 64],
    'n_a': [8, 16, 24, 32, 64],
    'n_steps': [3, 5, 7, 9, 11],
    'gamma': [1.0, 1.5, 2.0, 2.5],
    'lambda_sparse': [0, 1e-4, 1e-3, 1e-2]
}

# 데이터 numpy array로 변환
X_train_np = X_train.values
X_val_np = X_val.values

# 랜덤 서치
random_search = RandomizedSearchCV(
    tabnet, 
    param_dist, 
    n_iter=20, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=1, 
    verbose=2, 
    random_state=42
)

# 조기 중지 설정을 포함하여 모델 학습
random_search.fit(X_train_np, y_train_np, 
                  max_epochs=200, 
                  patience=10, 
                  eval_metric=['rmse'], 
                  eval_set=[(X_val_np, y_val_np)])

print("Best parameters found by Random Search:", random_search.best_params_)
print("Best cross-validation MSE by Random Search:", -random_search.best_score_)

# 검증 데이터에 대한 예측
y_pred_random = random_search.predict(X_val_np)

# 평가 지표 계산
mae = mean_absolute_error(y_val_np, y_pred_random)
mse = mean_squared_error(y_val_np, y_pred_random)
rmse = np.sqrt(mse)
r2 = r2_score(y_val_np, y_pred_random)

print("Validation MAE by Random Search:", mae)
print("Validation MSE by Random Search:", mse)
print("Validation RMSE by Random Search:", rmse)
print("Validation R² by Random Search:", r2)

# 랜덤 서치에서 최적의 파라미터 추출
best_params = random_search.best_params_


c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


Fitting 5 folds for each of 20 candidates, totalling 100 fits
epoch 0  | loss: 157.81074| val_0_rmse: 5.54347 |  0:04:56s
epoch 1  | loss: 32.94807| val_0_rmse: 5.36229 |  0:09:22s
epoch 2  | loss: 30.46575| val_0_rmse: 5.04574 |  0:13:49s
epoch 3  | loss: 28.62292| val_0_rmse: 5.00003 |  0:18:14s
epoch 4  | loss: 27.6523 | val_0_rmse: 5.18906 |  0:22:42s
epoch 5  | loss: 27.44149| val_0_rmse: 5.08769 |  0:27:08s
epoch 6  | loss: 27.35669| val_0_rmse: 4.93839 |  0:31:33s
epoch 7  | loss: 27.0736 | val_0_rmse: 4.85395 |  0:35:56s
epoch 8  | loss: 26.80343| val_0_rmse: 4.97945 |  0:40:19s
epoch 9  | loss: 26.50044| val_0_rmse: 4.92178 |  0:44:41s
epoch 10 | loss: 26.8736 | val_0_rmse: 4.85291 |  0:49:04s
epoch 11 | loss: 26.37037| val_0_rmse: 4.75167 |  0:53:26s
epoch 12 | loss: 26.21083| val_0_rmse: 4.88002 |  0:57:49s
epoch 13 | loss: 25.52691| val_0_rmse: 4.87282 |  1:02:11s
epoch 14 | loss: 25.19919| val_0_rmse: 4.94088 |  1:06:33s
epoch 15 | loss: 25.16069| val_0_rmse: 4.66147 |  1:

c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 210.43493| val_0_rmse: 6.98475 |  0:04:22s
epoch 1  | loss: 44.268  | val_0_rmse: 6.09675 |  0:08:45s
epoch 2  | loss: 35.42844| val_0_rmse: 5.75779 |  0:13:06s
epoch 3  | loss: 31.56798| val_0_rmse: 5.82487 |  0:17:28s
epoch 4  | loss: 29.9428 | val_0_rmse: 5.35753 |  0:21:50s
epoch 5  | loss: 29.79002| val_0_rmse: 5.19861 |  0:26:12s
epoch 6  | loss: 27.81145| val_0_rmse: 5.86507 |  0:30:34s
epoch 7  | loss: 28.08415| val_0_rmse: 4.86104 |  0:34:56s
epoch 8  | loss: 27.60313| val_0_rmse: 5.45555 |  0:39:18s
epoch 9  | loss: 26.96721| val_0_rmse: 5.11567 |  0:43:40s
epoch 10 | loss: 27.24981| val_0_rmse: 4.93857 |  0:48:01s
epoch 11 | loss: 26.73582| val_0_rmse: 12.53103|  0:52:24s
epoch 12 | loss: 26.49151| val_0_rmse: 5.77916 |  0:56:46s
epoch 13 | loss: 26.53871| val_0_rmse: 6.35827 |  1:01:08s
epoch 14 | loss: 26.26198| val_0_rmse: 7.52699 |  1:05:31s
epoch 15 | loss: 25.69684| val_0_rmse: 4.67852 |  1:09:55s
epoch 16 | loss: 27.29192| val_0_rmse: 6.61807 |  1:14:

KeyboardInterrupt: 

In [6]:
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import GridSearchCV
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# 데이터 준비 (train 데이터프레임이 이미 정의되어 있어야 합니다)
# 예시: train = pd.read_csv('train.csv')
X = train.drop('전력기상지수', axis=1)
y = train['전력기상지수']

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# numpy 배열로 변환
y_train_np = y_train.values.reshape(-1, 1)
y_val_np = y_val.values.reshape(-1, 1)
X_train_np = X_train.values
X_val_np = X_val.values

# 그리드 서치 파라미터 설정 (범위를 좁혀서)
param_grid = {
    'n_d': [8],  # 범위 좁힘
    'n_a': [8],  # 범위 좁힘
    'n_steps': [4],  # 범위 좁힘
    'gamma': [1.8, 2.0],  # 범위 좁힘
    'lambda_sparse': [0.0001, 0.001]  # 범위 좁힘
}

# TabNet 모델 생성 (새로운 객체로)
tabnet = TabNetRegressor()

# GridSearchCV 설정
grid_search = GridSearchCV(
    estimator=tabnet, 
    param_grid=param_grid, 
    cv=3, 
    scoring='neg_mean_squared_error',
    n_jobs=1,  # 가능한 모든 프로세서를 사용
    verbose=2
)

# 조기 중지 설정을 포함하여 모델 학습
grid_search.fit(X_train_np, y_train_np, 
                max_epochs=30,  # 최대 에포크 수를 줄임
                patience=5, 
                eval_metric=['rmse'], 
                eval_set=[(X_val_np, y_val_np)])

print("Best parameters found by Grid Search:", grid_search.best_params_)
print("Best cross-validation MSE by Grid Search:", -grid_search.best_score_)

# 검증 데이터에 대한 예측
y_pred_grid = grid_search.predict(X_val_np)

# 평가 지표 계산
mae = mean_absolute_error(y_val_np, y_pred_grid)
mse = mean_squared_error(y_val_np, y_pred_grid)
rmse = np.sqrt(mse)
r2 = r2_score(y_val_np, y_pred_grid)

print("Validation MAE by Grid Search:", mae)
print("Validation MSE by Grid Search:", mse)
print("Validation RMSE by Grid Search:", rmse)
print("Validation R² by Grid Search:", r2)

c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


Fitting 3 folds for each of 4 candidates, totalling 12 fits


c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


epoch 0  | loss: 178.59372| val_0_rmse: 7.00075 |  0:03:22s
epoch 1  | loss: 43.05306| val_0_rmse: 6.5761  |  0:06:40s
epoch 2  | loss: 36.88758| val_0_rmse: 6.31536 |  0:09:53s
epoch 3  | loss: 34.54943| val_0_rmse: 5.70431 |  0:13:05s
epoch 4  | loss: 35.51806| val_0_rmse: 5.75979 |  0:16:16s
epoch 5  | loss: 33.83595| val_0_rmse: 5.56334 |  0:19:27s
epoch 6  | loss: 32.72221| val_0_rmse: 5.6095  |  0:22:39s
epoch 7  | loss: 32.39753| val_0_rmse: 5.41635 |  0:25:52s
epoch 8  | loss: 32.08882| val_0_rmse: 5.16174 |  0:29:03s
epoch 9  | loss: 30.54395| val_0_rmse: 5.23892 |  0:32:15s
epoch 10 | loss: 29.56623| val_0_rmse: 5.28964 |  0:35:26s
epoch 11 | loss: 29.59462| val_0_rmse: 5.91515 |  0:38:37s
epoch 12 | loss: 29.66732| val_0_rmse: 5.57093 |  0:41:49s
epoch 13 | loss: 29.10019| val_0_rmse: 5.35448 |  0:45:01s

Early stopping occurred at epoch 13 with best_epoch = 8 and best_val_0_rmse = 5.16174
[CV] END gamma=1.8, lambda_sparse=0.0001, n_a=8, n_d=8, n_steps=4; total time=45.6min


c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


...

epoch 0  | loss: 159.60835| val_0_rmse: 5.55581 |  0:03:13s
epoch 1  | loss: 33.55218| val_0_rmse: 5.88878 |  0:06:26s
epoch 2  | loss: 30.5149 | val_0_rmse: 5.01947 |  0:09:41s
epoch 3  | loss: 28.95689| val_0_rmse: 5.38615 |  0:12:57s
epoch 4  | loss: 28.15463| val_0_rmse: 5.39253 |  0:16:10s
epoch 5  | loss: 27.55531| val_0_rmse: 4.83583 |  0:19:24s
epoch 6  | loss: 27.09008| val_0_rmse: 4.91575 |  0:22:39s
epoch 7  | loss: 26.72773| val_0_rmse: 4.95246 |  0:25:54s
epoch 8  | loss: 26.59884| val_0_rmse: 5.20878 |  0:29:08s
epoch 9  | loss: 27.00454| val_0_rmse: 5.1478  |  0:32:21s
epoch 10 | loss: 26.26479| val_0_rmse: 4.79957 |  0:35:35s
epoch 11 | loss: 25.52383| val_0_rmse: 4.72539 |  0:38:49s
epoch 12 | loss: 25.15853| val_0_rmse: 4.71981 |  0:42:04s
epoch 13 | loss: 24.92474| val_0_rmse: 4.80055 |  0:45:16s
epoch 14 | loss: 26.76759| val_0_rmse: 4.67992 |  0:48:30s
epoch 15 | loss: 26.28889| val_0_rmse: 5.21331 |  0:51:44s
epoch 16 | loss: 25.25662| val_0_rmse: 4.74106 |  0:54:

c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 166.00718| val_0_rmse: 5.66829 |  0:03:13s
epoch 1  | loss: 35.01276| val_0_rmse: 5.55814 |  0:06:27s
epoch 2  | loss: 31.24336| val_0_rmse: 5.26052 |  0:09:40s
epoch 3  | loss: 29.43727| val_0_rmse: 5.20063 |  0:12:54s
epoch 4  | loss: 28.30453| val_0_rmse: 5.1571  |  0:16:08s
epoch 5  | loss: 27.751  | val_0_rmse: 4.97945 |  0:19:21s
epoch 6  | loss: 27.15114| val_0_rmse: 4.96472 |  0:22:37s
epoch 7  | loss: 26.64624| val_0_rmse: 6.07173 |  0:25:52s
epoch 8  | loss: 26.36044| val_0_rmse: 5.32476 |  0:29:07s
epoch 9  | loss: 26.03431| val_0_rmse: 7.34531 |  0:32:21s
epoch 10 | loss: 25.80265| val_0_rmse: 5.29645 |  0:35:36s
epoch 11 | loss: 25.50959| val_0_rmse: 7.89704 |  0:38:51s

Early stopping occurred at epoch 11 with best_epoch = 6 and best_val_0_rmse = 4.96472
[CV] END gamma=2.0, lambda_sparse=0.001, n_a=8, n_d=8, n_steps=4; total time=39.3min


c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


ValueError: 
All the 12 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py", line 278, in fit
    self.feature_importances_ = self._compute_feature_importances(X_train)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py", line 759, in _compute_feature_importances
    M_explain, 

# 3

In [18]:
import torch
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor
import joblib

# 데이터 준비
X = train.drop('전력기상지수', axis=1)
y = train['전력기상지수']

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 목표 변수 numpy array로 변환 후 reshape
y_train_np = y_train.values.reshape(-1, 1)
y_val_np = y_val.values.reshape(-1, 1)

# 데이터 numpy array로 변환
X_train_np = X_train.values
X_val_np = X_val.values

# 데이터 타입과 모양 확인
print(type(X_train_np), X_train_np.shape)
print(type(y_train_np), y_train_np.shape)
print(type(X_val_np), X_val_np.shape)
print(type(y_val_np), y_val_np.shape)


# 하이퍼파라미터 설정
params = {
    'n_d': 8,
    'n_a': 8,
    'n_steps': 4,
    'gamma': 2.0,
    'lambda_sparse': 0.001
}

# TabNet Regressor 모델 정의 및 설정
model = TabNetRegressor(**params)

# 모델 학습
model.fit(
    X_train_np, y_train_np,
    max_epochs=7,  # 최대 에포크 수를 줄임
    patience=2, 
    eval_metric=['rmse'], 
    eval_set=[(X_val_np, y_val_np)]
)

<class 'numpy.ndarray'> (6074544, 95)
<class 'numpy.ndarray'> (6074544, 1)
<class 'numpy.ndarray'> (1518637, 95)
<class 'numpy.ndarray'> (1518637, 1)


c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 127.36815| val_0_rmse: 5.78054 |  0:04:35s
epoch 1  | loss: 34.25357| val_0_rmse: 5.8888  |  0:09:06s
epoch 2  | loss: 30.92995| val_0_rmse: 5.10062 |  0:13:38s
epoch 3  | loss: 29.65534| val_0_rmse: 5.21534 |  0:18:11s
epoch 4  | loss: 28.84016| val_0_rmse: 5.03884 |  0:22:43s
epoch 5  | loss: 28.03538| val_0_rmse: 4.74181 |  0:27:15s
epoch 6  | loss: 27.43432| val_0_rmse: 4.92455 |  0:31:48s
Stop training because you reached max_epochs = 7 with best_epoch = 5 and best_val_0_rmse = 4.74181


c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


KeyboardInterrupt: 

# 4

In [2]:
# model.save_model('best_tabnet_model')

# print(f"Best model saved to 'best_tabnet_model.zip'")
from pytorch_tabnet.tab_model import TabNetRegressor
# 최적 모델 로드 및 검증 데이터로 예측 수행
loaded_model = TabNetRegressor()
loaded_model.load_model(r'C:/Users/DC/OneDrive - 계명대학교/DC/2024/2024_날씨빅콘/코드/best_tabnet_model.zip')

c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [5]:
object_columns = test.select_dtypes(include=['object']).columns
print("Object 타입의 열:", object_columns)
test = pd.get_dummies(test, columns=object_columns)

Object 타입의 열: Index(['계절'], dtype='object')


In [6]:
# boolean 값을 정수형으로 변환
test = test.astype(float)

In [7]:
import numpy as np

X_test_np = test.values

# 최종 모델을 사용하여 test 데이터셋에 대한 예측 수행
y_test_pred = loaded_model.predict(X_test_np)

In [8]:
df = pd.read_csv(r'C:/Users/DC/OneDrive - 계명대학교/DC/2024/2024_날씨빅콘/제출데셋/electric_test.csv')

In [41]:
# def remove_nan(df, nan_value=-99):
#     for column in df.columns:
#         df = df[df[column] != nan_value]
#     return df


# df = remove_nan(df)

In [9]:
n_test = len(df)
n_pred = len(y_test_pred)

if n_test != n_pred:
    # 예측값 길이와 데이터프레임 길이 차이 계산
    diff = n_test - n_pred
    
    # 앞뒤 평균으로 채울 값을 생성
    y_test_pred_extended = list(y_test_pred)
    
    for i in range(diff):
        # 앞뒤 평균으로 결측값을 채움
        if i == 0:
            # 처음에 추가할 경우
            y_test_pred_extended.insert(0, (y_test_pred_extended[0] + y_test_pred_extended[1]) / 2)
        else:
            # 그 외의 경우
            y_test_pred_extended.append((y_test_pred_extended[-1] + y_test_pred_extended[-2]) / 2)

    y_test_pred = np.array(y_test_pred_extended)

# 데이터프레임에 예측값 추가
df['elect'] = y_test_pred

In [10]:
df

,NUM,TM,HH24,STN,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elect
0,2385,2023-01-01 01:00:00,1,303,7.8,61.5,6.7,0.0,4.2,6,1.0,95.594788
1,2385,2023-01-01 02:00:00,2,303,7.9,60.6,7.6,0.0,4.0,6,1.0,95.450401
2,2385,2023-01-01 03:00:00,3,303,8.2,61.9,8.7,0.0,4.1,6,1.0,95.739174
3,2385,2023-01-01 04:00:00,4,303,8.4,60.9,9.2,0.0,4.3,6,1.0,99.930763
4,2385,2023-01-01 05:00:00,5,303,8.5,60.9,9.5,0.0,4.3,6,1.0,98.406364
...,...,...,...,...,...,...,...,...,...,...,...,...
2838234,12322,2023-12-31 19:00:00,19,901,5.7,72.3,3.6,0.0,5.7,6,1.0,101.669601
2838235,12322,2023-12-31 20:00:00,20,901,5.6,70.8,3.2,0.0,5.8,6,1.0,101.669601
2838236,12322,2023-12-31 21:00:00,21,901,5.3,69.1,3.6,0.0,4.5,6,1.0,101.669601
2838237,12322,2023-12-31 22:00:00,22,901,5.1,70.6,3.0,0.0,5.1,6,1.0,101.669601


In [11]:
df.to_csv(r'C:/Users/DC/OneDrive - 계명대학교/DC/2024/2024_날씨빅콘/제출데셋/240223.csv', index=False)